In [32]:
import pandas as pd
import numpy as np
import toxic_funcs

In [2]:
#reading only first n for testing purposes
n = 100000
df = pd.read_csv("data/train.csv").set_index('id')
df = df.head(n)

## Preprocess

In [3]:
#running through the custom cleaner

clean_comments = []
for comment in df.comment_text.values:
    clean_comments.append(toxic_funcs.Cleaner(comment))


In [4]:
#slapping it onto df

df['cleaned'] = clean_comments

In [5]:
from keras.preprocessing.text import Tokenizer
#split texts into words

tokenizer = Tokenizer(num_words = 10000)

/Users/abhsharm/anaconda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
#
tokenizer.fit_on_texts(df.cleaned.values)

In [7]:
tokenized = tokenizer.texts_to_matrix(df.cleaned.values)

In [8]:
tokenizer

In [59]:
labels = ["toxic", "severe_toxic","obscene","threat","insult","identity_hate"]

label_strings = []
for index, row in df.iterrows():
   label_string = ''
   for label in labels:
       if row[label]:
           label_string += label
   label_strings.append(label_string)
label_code = []
for label in label_strings:
    label_code.append((toxic_funcs.PossibleCombination()).index(label))
label_strings_col = pd.Series(label_code)
df['combined_label'] = label_strings_col.values
df

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned,combined_label
id,,,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation. why the edits made under my usern...,63
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,daww. he matches this background colour im se...,63
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man im really not trying to edit war. its ...,63
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,. more. i cant make any real suggestions on im...,63
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,you sir are my hero. any chance you remember w...,63
00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,. congratulations from me as well use the too...,63
0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,cocksucker before you piss around on my work,7
00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,your vandalism to the matt shirvington article...,63
00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,sorry if the word nonsense was offensive to yo...,63


# Build labels and features

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

X = tokenized
y = to_categorical(np.array(df.combined_label))
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
'''X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)'''


'X_scaler = StandardScaler().fit(X_train)\nX_train_scaled = X_scaler.transform(X_train)\nX_test_scaled = X_scaler.transform(X_test)'

In [37]:
X_train

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [51]:
print(f"x-{X_train.shape} y-{y_train.shape}")
y_train

x-(75000, 10000) y-(75000, 64)


array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

# Build model

In [52]:
from keras.models import Sequential
from keras.layers import Dense

# Create a Neural Network model here
# YOUR CODE HERE
model = Sequential()
model.add(Dense(500,activation = "relu", input_dim = 10000))
model.add(Dense(70, activation = "relu"))
model.add(Dense(2, activation = "softmax"))

In [53]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)
model.fit(X_train, y_train, epochs = 10, verbose=1)

ValueError: Error when checking target: expected dense_3 to have shape (2,) but got array with shape (64,)

In [114]:
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Normal Neural Network Test - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network Test - Loss: 0.1565199643969536, Accuracy: 0.9916


In [89]:
#model.save("models/vanilla.h5")

In [116]:
from keras.models import load_model
model = load_model('models/vanilla.h5')

In [123]:
dummy = df[df.threat == 1]
dummy[dummy.toxic == 0].comment_text.values

array([ 'Please stop. If you continue to ignore our policies by introducing inappropriate pages to Wikipedia, you will be blocked.',
       '"\n\n Lock Down \n\nI don\'t know about the ""Bulgarian BULLSHIT season"", but don\'t be fooled from where exactly i connect. If i see you editing, or someone else you send, the DDA40X article one more time, you will spend the next few years in prison. I will arrange you a meeting with certain people for suspicions of terrorism. Yes. I can do that. A terror suspect nowadays, you can kiss your young life goodbye. If you file complaints and i\'m banned, you and your naked ass will have the whole time in the world, in the cell block, to think what went wrong. 87.252.167.254  "'], dtype=object)

64

In [143]:
df[df.cleaned.isin('fuck')]

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [str]

In [174]:
count = 0
for index, row in df.head(10000).iterrows():
    pred = 1 - round(model.predict(tokenizer.texts_to_matrix([row.cleaned]))[0][0],0)
    if pred != row.toxic:
        count += 1
        print(f"{index} - pred: {pred}  - act: {row.toxic}")
        print
        print(row.cleaned)
        print('=================================')
        if (index % 100) == 0:
            print('//////////////////////////////' + str(index) + '//////////////////////////////')

206 - pred: 0.0  - act: 1
give me a permanat block raseac.   remember me.   muslims dont you worry for these images they are trying to hurt us but they cant look these images are fake and have nothing to do with islam or our prophet even their article is so lame wikipedia is not an authentic source its an encyclopedia for the kids which has not authenticity anyone can edit or write whatever they think. just like raeasc you can see he does not knows what islam is and he talks about it so forget about wiki its lame they have no authenticity. i recommend they should add pissed christ in jessus article. they should. 
1169 - pred: 1.0  - act: 0
dont insert your as an administrator. you cant protect anything. 
1431 - pred: 0.0  - act: 1
.   camille paglia: sex positive feminist .  here are the supporting links you asked for you stupid twit.  positive_feminism.  if you are too lazy and full of yourself to read them in their entirety then i will just post the relevant sections here:.  camille 

3540 - pred: 0.0  - act: 1
.  it is not disruptive and does not appear to be attention seeking. these remarks are just argumentum ad hominem arguments nothing more. you have zero evidence of this supposed motive on my part and are just attacking me because we disagree on the article an unambiguous violation of wp:agf.  is there a particular article im working to improve.  there isnt.   how do you figure.  what is your basis for this accusation.  have you checked my edit history.  theres lots of articles im constantly working to improve as any look at my editing history shows which flies in the face of this idiotic remark of yours. there are entire lists of articles ive created and worked on seen on my user page including three articles i single handedly brought to ga status. the arthur adams (comics) article for example has been expanded and improved by me on an ongoing basis for the past three years and is currently undergoing a ga examination process. these inane comments by you do n

6088 - pred: 0.0  - act: 1
.  penis envy. from wikipedia the free encyclopedia. jump to: navigation search. for the crass album see penis envy (album). for the virgin  documentary see . envy.  penis envy in freudian psychoanalysis refers to the theorized reaction of a girl during her psychosexual development to the realization that she does not have a penis. freud considered this realization a defining moment in the development of gender and sexual identity for women. according to freud the parallel reaction in boys to the realization that girls do not have a penis is castration anxiety.  in contemporary culture the term is sometimes used symbolically or metaphorically to refer to the idea that women wish they had a penis or to refer to anxieties between men about the size of their genitals.  contents. hide.      *  freuds theory.           o  criticisms of freud’s theory.                    within psychoanalytic circles.                    feminist criticisms.     *  male penis envy. 

6506 - pred: 0.0  - act: 1
.  reply from colonelhenry. i have been pissed this week i admit. why.   first last week i get harassed by one hypocritical rule nazi editor taking things too far (summerphd) so when i pointed out her similar shortcomings and intransigence and said frankly this user is an asshole and explain why and state this is the kind of user that drives editors away from wikipedia i get blocked for  hours.  i say asshole once i get blocked.  she calls me a dick over a dozen times and gets warned.  go figure. i hate nitpicking rule nazis because they end up proving themselves to be hypocrites and unfortunately without fool other meek fools who tend to be admins tend to side with the rule nazi.   in the midst of that daniel case continued to plow ahead insisting on a rushed revision of alonj that no one else raised as a concern (so not immediately dire) after being asked first politely and respectfully several times to wait until after tfa and he still ignored it.  if some

7225 - pred: 0.0  - act: 1
.  anon user  behaviour .  today when i woke up i found a whole lot of personal attacks from anon user  against me.  messages such as:. this user is such a stupid and biased wikipedia editor. should be banned forever.  on my user page.  a blanking of my talk page whith the following message buaagh im sick only reading this user please. remove him from wiki.  that is fucking stupid and haves no sense.   that is a person with various nationalities not a nationality of spain . only maurice can say that.  in the nationalities of spain article.  wtf.   is not possible that maurice is still editing this article from a such biased and stupid pov.   in the catalonia article. (notice that i hadnt edit that article since   .  i ask you if you can do anything to stop this kind of behaviour with a ban from a user who has been attacking me since    and who im pretty sure is a sockpuppet from ). 
7300 - pred: 0.0  - act: 1
.  unfortunately you are an undeniable feeble pers

In [183]:
tot = df[df.toxic == 1].count()[0]
count/tot*100

3.6045314109165809

35

In [187]:
y_train

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [28]:
a = ["foo","bar","baz",'bar','any','much']
print(a.index("bar"))

1
